## Weather Data Collection


In [ ]:
import pandas as pd
import requests
import io
from google.colab import files
from datetime import datetime, timedelta


API_KEY = "7H4THT9QDNSJX23R5SX52VJDV"
location = "Istanbul,Turkey"
start_date = datetime.strptime("2025-03-14", "%Y-%m-%d")
end_date = datetime.today()


def fetch_weather_chunk(start, end):
    url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{location}/{start}/{end}"
    params = {
        "unitGroup": "metric",
        "key": API_KEY,
        "include": "days",
        "contentType": "csv"
    }
    response = requests.get(url, params=params)
    if response.status_code != 200:
        print(f"Error fetching data from {start} to {end}: {response.text}")
        return pd.DataFrame()
    df = pd.read_csv(io.StringIO(response.text))


    df["name"] = df["name"].str.encode("latin1").str.decode("utf-8")


    df = df[["datetime", "conditions", "tempmax", "tempmin", "humidity"]]
    df.columns = ["date", "weather_condition", "high_temp", "low_temp", "humidity"]

    return df


weather_frames = []
current = start_date

while current < end_date:

    next_month = (current.replace(day=1) + timedelta(days=32)).replace(day=1)
    chunk_end = min(end_date, next_month - timedelta(days=1))
    print(f"Fetching: {current.date()} to {chunk_end.date()}")
    chunk_df = fetch_weather_chunk(current.strftime("%Y-%m-%d"), chunk_end.strftime("%Y-%m-%d"))
    weather_frames.append(chunk_df)
    current = next_month


weather_data = pd.concat(weather_frames, ignore_index=True)

filename = "istanbul_weather_cleaned.csv"
weather_data.to_csv(filename, index=False)
files.download(filename)





Fetching: 2025-03-14 to 2025-03-31
Fetching: 2025-04-01 to 2025-04-25


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>